# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [34]:
city_reader=pd.read_csv("../output_data/my_cities.csv")

city_reader

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hambantota,6.12,81.12,78.35,75,44,21.07,LK,1580579350
1,kochki,54.34,80.48,-1.07,88,0,8.12,RU,1580579350
2,hilo,19.73,-155.09,69.80,73,90,6.93,US,1580579351
3,rikitea,-23.12,-134.97,79.30,79,87,15.21,PF,1580579351
4,todi,42.78,12.40,55.40,82,75,5.82,IT,1580579351
...,...,...,...,...,...,...,...,...,...
589,lata,40.16,-8.33,62.01,93,75,9.17,PT,1580579612
590,ostersund,63.18,14.64,33.01,100,75,5.82,SE,1580579612
591,rudbar,30.15,62.60,57.13,33,0,8.16,AF,1580579612
592,alice springs,-23.70,133.88,71.60,94,90,17.22,AU,1580579613


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [35]:
gmaps.configure(api_key=g_key)


In [36]:
locations = city_reader[["Lat", "Lng"]].astype(float)

humidity= city_reader["Humidity"].astype(float)

fig= gmaps.figure(zoom_level=2, center=(14.59,28.67))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
perf_weather=city_reader[(city_reader["Max Temp"]>=70) \
                      & (city_reader["Max Temp"]<80) \
                      & (city_reader["Wind Speed"]<10) \
                      & (city_reader["Cloudiness"]==0)] 

# perf_wind=perf_temp[city_reader["Wind Speed"]<10]

# no_cloud=city_reader[city_reader["Cloudiness"]==0]

# perf_weather=pd.DataFrame({"Perf Temp": [perf_temp], "Perf Wind": [perf_wind],
#                           "No Clouds": [no_cloud]})
    
    
# perf_temp

perf_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,avarua,-21.21,-159.78,77.00,94,0,2.24,CK,1580579365
258,ballina,-28.87,153.57,73.40,88,0,9.17,AU,1580579465
312,arlit,18.74,7.39,76.30,9,0,7.52,NE,1580579487
370,sao filipe,14.90,-24.50,79.47,46,0,9.33,CV,1580579515
394,salalah,17.02,54.09,75.20,53,0,4.70,OM,1580579349
582,los llanos de aridane,28.66,-17.92,75.20,41,0,8.05,ES,1580579609


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
perf_weather["Hotel Name"]=""

hotel_df=perf_weather

hotel_df

C:\Users\Jonpaul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
38,avarua,-21.21,-159.78,77.00,94,0,2.24,CK,1580579365,
258,ballina,-28.87,153.57,73.40,88,0,9.17,AU,1580579465,
312,arlit,18.74,7.39,76.30,9,0,7.52,NE,1580579487,
370,sao filipe,14.90,-24.50,79.47,46,0,9.33,CV,1580579515,
394,salalah,17.02,54.09,75.20,53,0,4.70,OM,1580579349,
582,los llanos de aridane,28.66,-17.92,75.20,41,0,8.05,ES,1580579609,


In [39]:
target_radius=5000
target_type="lodging"
target_coordinates="0,0"

params = {
"radius": target_radius,
"type": target_type,
"key": g_key
}

for i,location in hotel_df.iterrows():
    params["location"]=f"{location['Lat']},{location['Lng']}"

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[i,"Hotel Name"]=response["results"][0]["name"]
    except:
        print("No hotel found")

In [40]:
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
38,avarua,-21.21,-159.78,77.00,94,0,2.24,CK,1580579365,Paradise Inn
258,ballina,-28.87,153.57,73.40,88,0,9.17,AU,1580579465,Ramada Hotel & Suites by Wyndham Ballina Byron
312,arlit,18.74,7.39,76.30,9,0,7.52,NE,1580579487,Hôtel Telwa Bungalow
370,sao filipe,14.90,-24.50,79.47,46,0,9.33,CV,1580579515,Tortuga B&B
394,salalah,17.02,54.09,75.20,53,0,4.70,OM,1580579349,HAMDAN PLAZA HOTEL
582,los llanos de aridane,28.66,-17.92,75.20,41,0,8.05,ES,1580579609,Valle Aridane


In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perf_weather.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# Add marker layer ontop of heat map
# Hotel_Name = hotel_df["Hotel Name"].tolist()
markers = gmaps.marker_layer(locations,
                             info_box_content=[f"{name}" for name in hotel_info])
# Add the layer to the map
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))